# Wait Times

How are wait times related to the number of vehicles and their location? There is a hypothesis that the distance between one point and the nearest of N random points goes as the square root of N. This notebook is just some experiments to see if that holds, and to find out what the relationship is.

In [1]:
# imports and constants
# import random
import math
# Add the ridehail directory (..) to the system path to import ridehail modules
# See https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from ridehail.atom import City
DEFAULT_CITY_SIZE = 32
DEFAULT_SAMPLE_SIZE = 100

## City geometry

### Locations in a city

A city is a square grid of blocks with roads one "block" apart. The possible locations for vehicles and trips are at the intersections of the grid. For technical reasons the numbering starts at zero, so both the x and y coordinates are whole numbers from 0 to one less than the size of the city.

In [2]:
city_size = 4
city = City(city_size=city_size)
for i in range(6):
    print(f"A trip request is made at {city.set_trip_location()}")

A trip request is made at [2, 2]
A trip request is made at [2, 2]
A trip request is made at [0, 0]
A trip request is made at [0, 3]
A trip request is made at [0, 0]
A trip request is made at [1, 1]


## Distances in the city

One peculiarity of this model is that vehicles can drive off one edge and appear immediately at the opposite edge.

This setup simplifies things in that each location is identical to any other location, and a city is defined solely by its size. Topologically, this makes the city a doughnut shape which seems unlikely. It may be better to think that any actual fixed city or other region has boundaries, and vehicles can always cross those boundaries, leaving and entering the city. So you can think of this setup as a steady state in which vehicles are entering and leaving at the same rate.

Here is a trip request made at a random location (printed) and the distances to a set of vehicles scattered around the city. You can do the arithmetic to see how the distances work.

In [3]:
sample_size = 6
city_size = 4
city = City(city_size=city_size)
trip_request_point = city.set_trip_location()
print(f"A trip request is made at {trip_request_point}")
for i in range(sample_size):
    vehicle_location = city.set_trip_location()
    print(f"The vehicle at {vehicle_location} is {city.distance(trip_request_point, vehicle_location)} blocks away")

A trip request is made at [1, 1]
The vehicle at [0, 2] is 2 blocks away
The vehicle at [2, 1] is 1 blocks away
The vehicle at [1, 1] is 0 blocks away
The vehicle at [3, 3] is 4 blocks away
The vehicle at [1, 0] is 1 blocks away
The vehicle at [2, 0] is 2 blocks away


The next cell shows that the average distance between two randomly selected points is half the city size. It won't be exact of course, but for large sample sizes it is increasingly close.

In [4]:
sample_size = 100000
city_size = 16
city = City(city_size=city_size)
average_distance = 0.0
for i in range(sample_size):
    average_distance += city.distance(city.set_trip_location(), city.set_trip_location())
average_distance = average_distance / sample_size
print(
    f"For a city of {city_size} by {city_size} blocks, the average distance "
    f"between two randomly-selected points is {average_distance:.2f} blocks")

For a city of 16 by 16 blocks, the average distance between two randomly-selected points is 8.00 blocks


## Wait times

When a trip request is made, there may be a number of available vehicles scattered around the city. The trip is assigned to the nearest vehicle (if there are several an equal distance away, one of those is picked).

The city block can be thought of as a distance or a time. The wait time is the number of blocks from the assigned vehicle to the request location.

The question is, how does the average wait time for a request change with the number of available vehicles? And if there are several requests made at once, what is the average wait time for those requests?

Here are a few functions to set a number of vehicle locations in a city at once, and to get the wait time to a location from among those vehicles. Also, as we are dealing with randomness and averages, there is a function that repeatedly sets a trip location, computes the wait time for a set of N vehicles, and then gives the average.

In [5]:
def set_vehicle_locations(vehicle_count, city):
    vehicle_locations = []
    for vehicle in range(0,vehicle_count):
        vehicle_locations.append(city.set_trip_location())
    return vehicle_locations

def get_wait_time(request_location, vehicle_locations, city, allow_zero_wait_times=True):
    if allow_zero_wait_times:
        vehicle_distances = [city.distance(request_location, vehicle) for vehicle in vehicle_locations]
    else:
        vehicle_distances = [max(city.distance(request_location, vehicle), 1) for vehicle in vehicle_locations]
    min_distance = min(vehicle_distances)
    mean_distance = float(sum(vehicle_distances))/float(len(vehicle_distances))
    return (min_distance, 
            mean_distance)

def mean_request_wait_time(vehicle_count, city, sample_size=DEFAULT_SAMPLE_SIZE):
    mean_wait_time = 0
    mean_all_distance = 0
    for i in range(0,sample_size):
        request_location = city.set_trip_location()
        vehicle_locations = set_vehicle_locations(vehicle_count, city)
        (wait_time, mean_distance) = get_wait_time(request_location, vehicle_locations, city)
        mean_wait_time += wait_time
        mean_all_distance += mean_distance
    mean_wait_time = float(mean_wait_time) / float(sample_size)
    mean_all_distance = float(mean_all_distance) / float(sample_size)
    return (mean_wait_time, mean_all_distance)

## Wait times for a single request

Now we can carry out some experiments. Here you can set a sample size, a city size, and a range of vehicle counts, and print out the wait time.

I had seen some suggestion that the wait time should be like the square root of the number of cars. You can see here that for this city, that's not too bad. An estimate is that the wait time for a single request is:

$W = L / \sqrt(N)$

where L is the average separation of the $N$ vehicles.

This seems to underestimate by about 20% at small vehicle counts, and overestimate at large vehicle counts (when the wait time is significantly less than one block).

In [6]:
sample_size = 1000
city_size = 32
city=City(city_size)
min_vehicle_count = 100
max_vehicle_count = 1000
vehicle_count_increment = 100
vehicle_count_list = [city_size * x for x in range(1,16)]
for i in range(1,5):
    vehicle_count_list.insert(0, i)
# vehicle_count_list += list(range(min_vehicle_count, max_vehicle_count + 1, vehicle_count_increment))
print(f"Vehicles | Mean distance | Wait Time | Estimate | Error |")
for vehicle_count in vehicle_count_list:
    (mean_wait_time, mean_all_distance) = mean_request_wait_time(vehicle_count, city, sample_size)
    estimate = mean_all_distance / (math.pow((vehicle_count), 0.5))
    error = 100.0 * (estimate - mean_wait_time) / mean_wait_time
    print(f"{vehicle_count:8d} | {mean_all_distance:13.2f} | {mean_wait_time:9.2f} | "
          f"{estimate:8.2f} | {error:4.0f}% |")

Vehicles | Mean distance | Wait Time | Estimate | Error |
       4 |         16.01 |      9.12 |     8.01 |  -12% |
       3 |         15.99 |     10.24 |     9.23 |  -10% |
       2 |         15.90 |     12.08 |    11.25 |   -7% |
       1 |         16.07 |     16.07 |    16.07 |    0% |
      32 |         15.95 |      3.36 |     2.82 |  -16% |
      64 |         15.99 |      2.36 |     2.00 |  -15% |
      96 |         16.02 |      1.95 |     1.63 |  -16% |
     128 |         15.99 |      1.65 |     1.41 |  -14% |
     160 |         16.02 |      1.49 |     1.27 |  -15% |
     192 |         15.99 |      1.26 |     1.15 |   -8% |
     224 |         15.99 |      1.23 |     1.07 |  -13% |
     256 |         16.00 |      1.10 |     1.00 |   -9% |
     288 |         15.99 |      1.00 |     0.94 |   -5% |
     320 |         16.00 |      0.94 |     0.89 |   -5% |
     352 |         15.98 |      0.90 |     0.85 |   -6% |
     384 |         16.01 |      0.84 |     0.82 |   -3% |
     416 |    

## Average wait times with multiple requests

Now suppose $R$ trip requests are made at once, and there are $N$ vehicles available to grant them.

Assign the trips sequentially: the first trip should have an average wait time of $L/\sqrt(N)$, the second $L/\sqrt(N-1)$, and so on until all $R$ trips are assigned. The average wait time should therefore be:

$<W> = (L/R) \sum_{i=1}^{R} 1/\sqrt(N - i)$

This section assumes we can repeat with smaller numbers of vehicles, rather than removing a specific vehicle each time.

In [11]:
sample_size = 1000
city_size = 16
request_count = 24
vehicle_count = 240
city=City(city_size)
mean_over_requests_wait_time = 0
mean_over_requests_distance = 0
mean_over_requests_wait_time_estimate = 0
print(f"Request | Vehicles | Wait Time | Estimate | Error |")
for r in range(request_count):
    if r >= vehicle_count:
        print("R is bigger than vehicle_count")
        break
    # Get the (sampled) wait time for the request with (vehicle_count - r) available vehicles
    (mean_wait_time, mean_all_distance) = mean_request_wait_time(vehicle_count - r, city, sample_size)
    mean_over_requests_wait_time += mean_wait_time
    mean_over_requests_distance += mean_all_distance
    wait_time_estimate = mean_all_distance * math.pow((vehicle_count - r), -0.5)
    mean_over_requests_wait_time_estimate += wait_time_estimate
    error = 100.0 * (wait_time_estimate - mean_wait_time) / mean_wait_time
    print(f"{r:7d} | {vehicle_count - r:8d} | {mean_wait_time:9.2f} | "
          f"{wait_time_estimate:8.2f} | {error:4.0f}% |")
mean_over_requests_wait_time = mean_over_requests_wait_time / float(request_count)
mean_over_requests_distance = mean_over_requests_distance / float(request_count)
mean_over_requests_wait_time_estimate /= request_count
error = 100.0 * (mean_over_requests_wait_time_estimate - mean_over_requests_wait_time)/ mean_over_requests_wait_time
print("-" * 80)
print(f"Request count={request_count:d} | Vehicle_count={vehicle_count:d} |"
    f" Mean wait time={mean_over_requests_wait_time:.2f} | Estimate={mean_over_requests_wait_time_estimate:.2f} |"
     f" Error={error:.0f}%")

Request | Vehicles | Wait Time | Estimate | Error |
      0 |      240 |      0.38 |     0.52 |   35% |
      1 |      239 |      0.39 |     0.52 |   32% |
      2 |      238 |      0.42 |     0.52 |   23% |
      3 |      237 |      0.41 |     0.52 |   28% |
      4 |      236 |      0.42 |     0.52 |   25% |
      5 |      235 |      0.42 |     0.52 |   23% |
      6 |      234 |      0.39 |     0.52 |   34% |
      7 |      233 |      0.43 |     0.52 |   22% |
      8 |      232 |      0.41 |     0.53 |   29% |
      9 |      231 |      0.43 |     0.53 |   23% |
     10 |      230 |      0.43 |     0.53 |   22% |
     11 |      229 |      0.43 |     0.53 |   22% |
     12 |      228 |      0.43 |     0.53 |   23% |
     13 |      227 |      0.42 |     0.53 |   27% |
     14 |      226 |      0.43 |     0.53 |   23% |
     15 |      225 |      0.43 |     0.53 |   23% |
     16 |      224 |      0.42 |     0.53 |   26% |
     17 |      223 |      0.42 |     0.54 |   28% |
     18 |   

In [8]:
city_size = 14
city=City(city_size)
sample_size = 100000
dist = 0.0
for i in range(sample_size):
    dist += city.distance(city.set_trip_location(), city.set_trip_location())
print(dist/float(sample_size))

7.00251


In [9]:
city_size = 8
city=City(city_size)
print(city.distance((1,1), (7, 2)))
for i in range(10):
    loc = city.set_trip_location()
    print(loc, city.distance((1,1), loc))

3
[4, 7] 5
[0, 1] 1
[0, 2] 2
[0, 5] 5
[2, 5] 5
[0, 4] 4
[2, 5] 5
[6, 2] 4
[5, 3] 6
[4, 7] 5
